In [237]:
import os
import kagglehub
import shutil
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import geopandas as gpd
import pandas as pd
import numpy as np

In [238]:
DOWNLOAD_ROOT = "mabusalah/brent-oil-prices"
DATA_DIR = os.path.join("datasets", "oilprices")

def fetch_data(housing_url=DOWNLOAD_ROOT, housing_path=DATA_DIR):
    if not os.path.isdir(housing_path):
        os.makedirs(housing_path)

    downloaded_path = kagglehub.dataset_download(housing_url)

    if os.path.isdir(downloaded_path):
        for file_name in os.listdir(downloaded_path):
            shutil.move(os.path.join(downloaded_path, file_name), housing_path)
    else:
        shutil.move(downloaded_path, housing_path)

    print("Dataset descargado en:", housing_path)

fetch_data()

Dataset descargado en: datasets\oilprices


In [239]:
def load_archive_data(housing_path=DATA_DIR, encoding='utf-8'):
    csv_path = os.path.join(housing_path, "BrentOilPrices.csv")
    return pd.read_csv(csv_path, sep=',', encoding=encoding,index_col=0)

try:
    oil_price_data = load_archive_data()
except UnicodeDecodeError:
    try:
        oil_price_data = load_archive_data(encoding='ISO-8859-1')
    except UnicodeDecodeError:
        oil_price_data = load_archive_data(encoding='Windows-1252')

oil_price_data.head()

,Price
Date,
20-May-87,18.63
21-May-87,18.45
22-May-87,18.55
25-May-87,18.60
26-May-87,18.63


In [240]:
oil_price_data = oil_price_data.reset_index()
oil_price_data.head()

,Date,Price
0,20-May-87,18.63
1,21-May-87,18.45
2,22-May-87,18.55
3,25-May-87,18.60
4,26-May-87,18.63


In [271]:
fig = px.line(oil_price_data, 
              x='Date', 
              y='Price',
              labels={'Price':'Price (USD)',
                      'Date':'Date'})

fig.update_layout(
    title=dict(
        text="Brent Oil Prices",
        font=dict(size=20),
        x=0.5,  # Centra el título horizontalmente
        xanchor='center',  # Ancla el título en su centro
        yanchor='top'  # Ancla el título en la parte superior
    )
)
fig.show()

In [242]:
dates_oil_price_data = oil_price_data.copy()
dates_oil_price_data['Date'] = pd.to_datetime(dates_oil_price_data['Date'])
dates_oil_price_data['Year'] = dates_oil_price_data['Date'].dt.year
dates_oil_price_data['Month'] = dates_oil_price_data['Date'].dt.month_name(locale='en')
dates_oil_price_data.head()

C:\Users\franf\AppData\Local\Temp\ipykernel_19652\453004550.py:2: UserWarning:

Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.



,Date,Price,Year,Month
0,1987-05-20,18.63,1987,May
1,1987-05-21,18.45,1987,May
2,1987-05-22,18.55,1987,May
3,1987-05-25,18.60,1987,May
4,1987-05-26,18.63,1987,May


In [243]:
mouthly_oil_price_data = dates_oil_price_data[['Month','Price']].groupby(['Month']).mean().reset_index()
mouthly_oil_price_data.head(12)

,Month,Price
0,April,48.795262
1,August,50.008715
2,December,45.609861
3,February,46.975216
4,January,46.115007
5,July,49.879719
6,June,49.656970
7,March,48.370495
8,May,50.071047
9,November,46.867992


In [270]:
px.defaults.template = "ggplot2"
fig = px.bar(mouthly_oil_price_data,
                y='Month',
                x='Price',
                labels={'Price':'Price (USD)',
                        'Month':'Month'})
fig.update_layout(
    title=dict(
        text="Monthly Brent Oil Prices",
        font=dict(size=20),
        x=0.5, 
        xanchor='center',
        yanchor='top'
    ),
    width=600,
    height=400 
)
fig.update_yaxes(categoryorder='total ascending')
fig.update_xaxes(range=[45, 51])
fig.show()

In [245]:
years_oil_price_data = dates_oil_price_data[['Year','Price']].groupby(['Year']).mean().reset_index()
years_oil_price_data.head()

,Year,Price
0,1987,18.525813
1,1988,14.905412
2,1989,18.228228
3,1990,23.761445
4,1991,20.041128


In [267]:
x = years_oil_price_data['Year']
y = years_oil_price_data['Price']

import plotly.graph_objects as go

fig = px.bar(years_oil_price_data,
             x='Year',
             y='Price',
                labels={'Price':'Price (USD)',
                        'Year':'Year'})
fig.add_trace(
    go.Scatter(
        x=x,
        y=y,
        mode='lines',
        name='Tendencia',
        showlegend=False,
        line=dict(color='black',
                  width=2,
                  dash='dot') 
    )
)

fig.update_layout(
    title=dict(
        text="Yearly Brent Oil Prices",
        font=dict(size=20),
        x=0.5, 
        xanchor='center',
        yanchor='top'
    ),
    width=1000,
    height=400 
)

fig.show()